In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df=sb.load_dataset("tips")
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
df.describe

<bound method NDFrame.describe of      total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
239       29.03  5.92    Male     No   Sat  Dinner     3
240       27.18  2.00  Female    Yes   Sat  Dinner     2
241       22.67  2.00    Male    Yes   Sat  Dinner     2
242       17.82  1.75    Male     No   Sat  Dinner     2
243       18.78  3.00  Female     No  Thur  Dinner     2

[244 rows x 7 columns]>

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null category
smoker        244 non-null category
day           244 non-null category
time          244 non-null category
size          244 non-null int64
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [5]:
df.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


In [6]:
df1=df.groupby(['day','size']).sum()
df1

total_bill     tip
day  size                    
Thur 1          10.07    1.83
     2         727.53  117.24
     3          76.64   10.77
     4         149.75   21.09
     5          41.19    5.00
     6          91.15   15.90
Fri  1           8.58    1.92
     2         261.15   42.31
     3          15.98    3.00
     4          40.17    4.73
     5            NaN     NaN
     6            NaN     NaN
Sat  1          10.32    2.00
     2         892.37  133.43
     3         459.17   68.36
     4         388.39   53.61
     5          28.15    3.00
     6            NaN     NaN
Sun  1            NaN     NaN
     2         684.84  109.86
     3         332.76   46.81
     4         480.39   73.58
     5          81.00   12.14
     6          48.17    5.00

In [7]:
df2=df1.dropna(axis=0)
df2

total_bill     tip
day  size                    
Thur 1          10.07    1.83
     2         727.53  117.24
     3          76.64   10.77
     4         149.75   21.09
     5          41.19    5.00
     6          91.15   15.90
Fri  1           8.58    1.92
     2         261.15   42.31
     3          15.98    3.00
     4          40.17    4.73
Sat  1          10.32    2.00
     2         892.37  133.43
     3         459.17   68.36
     4         388.39   53.61
     5          28.15    3.00
Sun  2         684.84  109.86
     3         332.76   46.81
     4         480.39   73.58
     5          81.00   12.14
     6          48.17    5.00

In [8]:
df.replace({'sex':{'Male':0,'Female':1},'smoker':{'No':0,'Yes':1}},inplace=True)
df.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,0,0,Sat,Dinner,3
240,27.18,2.00,1,1,Sat,Dinner,2
241,22.67,2.00,0,1,Sat,Dinner,2
242,17.82,1.75,0,0,Sat,Dinner,2
243,18.78,3.00,1,0,Thur,Dinner,2


In [9]:
days=pd.get_dummies(df['day'],drop_first=True)
df=pd.concat([df,days],axis=1)
df.tail()

,total_bill,tip,sex,smoker,day,time,size,Fri,Sat,Sun
239,29.03,5.92,0,0,Sat,Dinner,3,0,1,0
240,27.18,2.00,1,1,Sat,Dinner,2,0,1,0
241,22.67,2.00,0,1,Sat,Dinner,2,0,1,0
242,17.82,1.75,0,0,Sat,Dinner,2,0,1,0
243,18.78,3.00,1,0,Thur,Dinner,2,0,0,0


In [10]:
times=pd.get_dummies(df['time'],drop_first=True)
df=pd.concat([df,times],axis=1)
df.head()

,total_bill,tip,sex,smoker,day,time,size,Fri,Sat,Sun,Dinner
0,16.99,1.01,1,0,Sun,Dinner,2,0,0,1,1
1,10.34,1.66,0,0,Sun,Dinner,3,0,0,1,1
2,21.01,3.50,0,0,Sun,Dinner,3,0,0,1,1
3,23.68,3.31,0,0,Sun,Dinner,2,0,0,1,1
4,24.59,3.61,1,0,Sun,Dinner,4,0,0,1,1


In [15]:
x=df[['total_bill','tip','sex','smoker','Fri','Sat','Sun','Dinner']]
y=df[['size']]

In [16]:
print(x,y)

     total_bill   tip  sex  smoker  Fri  Sat  Sun  Dinner
0         16.99  1.01    1       0    0    0    1       1
1         10.34  1.66    0       0    0    0    1       1
2         21.01  3.50    0       0    0    0    1       1
3         23.68  3.31    0       0    0    0    1       1
4         24.59  3.61    1       0    0    0    1       1
..          ...   ...  ...     ...  ...  ...  ...     ...
239       29.03  5.92    0       0    0    1    0       1
240       27.18  2.00    1       1    0    1    0       1
241       22.67  2.00    0       1    0    1    0       1
242       17.82  1.75    0       0    0    1    0       1
243       18.78  3.00    1       0    0    0    0       1

[244 rows x 8 columns]      size
0       2
1       3
2       3
3       2
4       4
..    ...
239     3
240     2
241     2
242     2
243     2

[244 rows x 1 columns]


In [17]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [18]:
svclassifier=SVC(kernel='linear')
svclassifier.fit(x_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [19]:
y_pred=svclassifier.predict(x_test)

In [20]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[ 0  1  0  0  0]
 [ 0 32  0  0  0]
 [ 0  3  0  0  0]
 [ 0  7  0  3  0]
 [ 0  2  0  1  0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.71      1.00      0.83        32
           3       0.00      0.00      0.00         3
           4       0.75      0.30      0.43        10
           5       0.00      0.00      0.00         3

    accuracy                           0.71        49
   macro avg       0.29      0.26      0.25        49
weighted avg       0.62      0.71      0.63        49



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
